In [1]:
import pandas as pd
import numpy as np
import json
import gzip
import os
import os.path
from os import listdir
import multiprocessing
from tqdm import tqdm
import mmap
from multiprocessing import Pool
import seaborn as sns
import matplotlib.pyplot as plt
import py_entitymatching as em
import py_stringmatching as sm
import scipy
from scipy.io import arff
from sklearn import preprocessing
#from sklearn.model_selection import train_test_split
from skmultilearn.model_selection import iterative_train_test_split
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit

In [2]:
#read table to dataframe and create train/test split with equal distribution of labels within table
#example small book tables
small = pd.read_csv('../../src/data/schemafiltereddata/small_tables_more_than_20.csv')
#small_books = pd.DataFrame()
#small_books = small[small['Class']=='Book']
small = small.drop(columns={'Unnamed: 0'})
pd.set_option('display.max_columns', None)
#small_books
small

,filename,Class,num_rows,num_columns,num_columns_selected,num_NAs,percentage_col_NA,percentage_table_NA,sum_NA_sel_col,Book_aggregaterating,Book_author,Book_bookedition,Book_bookformat,Book_datepublished,Book_genre,Book_inlanguage,Book_isbn,Book_name,Book_numberofpages,Book_offers,Book_publisher,CreativeWork_aggregaterating,CreativeWork_alternativeheadline,CreativeWork_articlebody,CreativeWork_author,CreativeWork_commentcount,CreativeWork_copyrightholder,CreativeWork_copyrightyear,CreativeWork_creator,CreativeWork_datecreated,CreativeWork_datemodified,CreativeWork_datepublished,CreativeWork_genre,CreativeWork_headline,CreativeWork_inlanguage,CreativeWork_interactioncount,CreativeWork_interactionstatistic,CreativeWork_interactiontype,CreativeWork_keywords,CreativeWork_mainentityofpage,CreativeWork_name,CreativeWork_offers,CreativeWork_publisher,CreativeWork_version,Event_doortime,Event_duration,Event_enddate,Event_eventattendancemode,Event_eventstatus,Event_location,Event_name,Event_organizer,Event_performer,Event_performers,Event_typicalagerange,Hotel_address,Hotel_aggregaterating,Hotel_email,Hotel_geo,Hotel_name,Hotel_pricerange,Hotel_starrating,Hotel_telephone,LocalBusiness_address,LocalBusiness_addresslocality,LocalBusiness_addressregion,LocalBusiness_aggregaterating,LocalBusiness_areaserved,LocalBusiness_citystatezip,LocalBusiness_contactpoint,LocalBusiness_email,LocalBusiness_faxnumber,LocalBusiness_geo,LocalBusiness_hasmap,LocalBusiness_legalname,LocalBusiness_location,LocalBusiness_name,LocalBusiness_openinghours,LocalBusiness_openinghoursspecification,LocalBusiness_paymentaccepted,LocalBusiness_postalcode,LocalBusiness_pricerange,LocalBusiness_review,LocalBusiness_sameas,LocalBusiness_streetaddress,LocalBusiness_telephone,MusicAlbum_byartist,MusicAlbum_name,MusicAlbum_numtracks,MusicAlbum_track,MusicRecording_byartist,MusicRecording_datecreated,MusicRecording_datepublished,MusicRecording_duration,MusicRecording_genre,MusicRecording_inalbum,MusicRecording_name,Person_additionalname,Person_address,Person_affiliation,Person_birthdate,Person_birthplace,Person_contactpoint,Person_deathdate,Person_email,Person_familyname,Person_faxnumber,Person_gender,Person_givenname,Person_height,Person_homelocation,Person_jobtitle,Person_knowslanguage,Person_makesoffer,Person_memberof,Person_nationality,Person_sameas,Person_telephone,Person_weight,Person_worklocation,Person_worksfor,Place_geo,Place_name,Place_openinghoursspecification,Place_sameas,Place_telephone,Product_additionalproperty,Product_address,Product_aggregaterating,Product_alternatename,Product_audience,Product_author,Product_availability,Product_bestrating,Product_brand,Product_breadcrumb,Product_category,Product_color,Product_condition,Product_datepublished,Product_depth,Product_disambiguatingdescription,Product_geo,Product_gtin,Product_gtin12,Product_gtin13,Product_gtin14,Product_gtin8,Product_height,Product_identifier,Product_isrelatedto,Product_issimilarto,Product_itemcondition,Product_mainentityofpage,Product_manufacturer,Product_material,Product_model,Product_mpn,Product_name,Product_offers,Product_price,Product_pricecurrency,Product_ratingvalue,Product_releasedate,Product_review,Product_reviewcount,Product_reviews,Product_shop-currency,Product_title,Product_weight,Product_width,Product_worstrating,Recipe_aggregaterating,Recipe_author,Recipe_cookingmethod,Recipe_cooktime,Recipe_datemodified,Recipe_datepublished,Recipe_headline,Recipe_ingredients,Recipe_ispartof,Recipe_keywords,Recipe_mainentityofpage,Recipe_name,Recipe_nutrition,Recipe_performtime,Recipe_preptime,Recipe_publisher,Recipe_recipecategory,Recipe_recipecuisine,Recipe_recipeingredient,Recipe_recipeinstructions,Recipe_recipeyield,Recipe_review,Recipe_suitablefordiet,Recipe_totaltime,Restaurant_acceptsreservations,Restaurant_address,Restaurant_aggregaterating,Restaurant_geo,Restaurant_hasmenu,Restaurant_menu,Restaurant_name,Restaurant_openinghours,Restaurant_openinghoursspecification,Restaurant_pricerange,Res

In [5]:
small['id'] = small.index
small

,filename,Class,num_rows,num_columns,num_columns_selected,num_NAs,percentage_col_NA,percentage_table_NA,sum_NA_sel_col,Book_aggregaterating,Book_author,Book_bookedition,Book_bookformat,Book_datepublished,Book_genre,Book_inlanguage,Book_isbn,Book_name,Book_numberofpages,Book_offers,Book_publisher,CreativeWork_aggregaterating,CreativeWork_alternativeheadline,CreativeWork_articlebody,CreativeWork_author,CreativeWork_commentcount,CreativeWork_copyrightholder,CreativeWork_copyrightyear,CreativeWork_creator,CreativeWork_datecreated,CreativeWork_datemodified,CreativeWork_datepublished,CreativeWork_genre,CreativeWork_headline,CreativeWork_inlanguage,CreativeWork_interactioncount,CreativeWork_interactionstatistic,CreativeWork_interactiontype,CreativeWork_keywords,CreativeWork_mainentityofpage,CreativeWork_name,CreativeWork_offers,CreativeWork_publisher,CreativeWork_version,Event_doortime,Event_duration,Event_enddate,Event_eventattendancemode,Event_eventstatus,Event_location,Event_name,Event_organizer,Event_performer,Event_performers,Event_typicalagerange,Hotel_address,Hotel_aggregaterating,Hotel_email,Hotel_geo,Hotel_name,Hotel_pricerange,Hotel_starrating,Hotel_telephone,LocalBusiness_address,LocalBusiness_addresslocality,LocalBusiness_addressregion,LocalBusiness_aggregaterating,LocalBusiness_areaserved,LocalBusiness_citystatezip,LocalBusiness_contactpoint,LocalBusiness_email,LocalBusiness_faxnumber,LocalBusiness_geo,LocalBusiness_hasmap,LocalBusiness_legalname,LocalBusiness_location,LocalBusiness_name,LocalBusiness_openinghours,LocalBusiness_openinghoursspecification,LocalBusiness_paymentaccepted,LocalBusiness_postalcode,LocalBusiness_pricerange,LocalBusiness_review,LocalBusiness_sameas,LocalBusiness_streetaddress,LocalBusiness_telephone,MusicAlbum_byartist,MusicAlbum_name,MusicAlbum_numtracks,MusicAlbum_track,MusicRecording_byartist,MusicRecording_datecreated,MusicRecording_datepublished,MusicRecording_duration,MusicRecording_genre,MusicRecording_inalbum,MusicRecording_name,Person_additionalname,Person_address,Person_affiliation,Person_birthdate,Person_birthplace,Person_contactpoint,Person_deathdate,Person_email,Person_familyname,Person_faxnumber,Person_gender,Person_givenname,Person_height,Person_homelocation,Person_jobtitle,Person_knowslanguage,Person_makesoffer,Person_memberof,Person_nationality,Person_sameas,Person_telephone,Person_weight,Person_worklocation,Person_worksfor,Place_geo,Place_name,Place_openinghoursspecification,Place_sameas,Place_telephone,Product_additionalproperty,Product_address,Product_aggregaterating,Product_alternatename,Product_audience,Product_author,Product_availability,Product_bestrating,Product_brand,Product_breadcrumb,Product_category,Product_color,Product_condition,Product_datepublished,Product_depth,Product_disambiguatingdescription,Product_geo,Product_gtin,Product_gtin12,Product_gtin13,Product_gtin14,Product_gtin8,Product_height,Product_identifier,Product_isrelatedto,Product_issimilarto,Product_itemcondition,Product_mainentityofpage,Product_manufacturer,Product_material,Product_model,Product_mpn,Product_name,Product_offers,Product_price,Product_pricecurrency,Product_ratingvalue,Product_releasedate,Product_review,Product_reviewcount,Product_reviews,Product_shop-currency,Product_title,Product_weight,Product_width,Product_worstrating,Recipe_aggregaterating,Recipe_author,Recipe_cookingmethod,Recipe_cooktime,Recipe_datemodified,Recipe_datepublished,Recipe_headline,Recipe_ingredients,Recipe_ispartof,Recipe_keywords,Recipe_mainentityofpage,Recipe_name,Recipe_nutrition,Recipe_performtime,Recipe_preptime,Recipe_publisher,Recipe_recipecategory,Recipe_recipecuisine,Recipe_recipeingredient,Recipe_recipeinstructions,Recipe_recipeyield,Recipe_review,Recipe_suitablefordiet,Recipe_totaltime,Restaurant_acceptsreservations,Restaurant_address,Restaurant_aggregaterating,Restaurant_geo,Restaurant_hasmenu,Restaurant_menu,Restaurant_name,Restaurant_openinghours,Restaurant_openinghoursspecification,Restaurant_pricerange,Res

In [ ]:
small_books['id'] = small_books.index
#small_books

In [21]:
hotel_col = list([col for col in small if col.startswith('Hotel_')])
hotel_col

['Hotel_address',
 'Hotel_aggregaterating',
 'Hotel_email',
 'Hotel_geo',
 'Hotel_name',
 'Hotel_pricerange',
 'Hotel_starrating',
 'Hotel_telephone']

In [22]:
hotel_grouped = small.groupby('Class').agg({'Hotel_address':'sum','Hotel_aggregaterating':'sum','Hotel_email':'sum','Hotel_geo':'sum','Hotel_name':'sum', 'Hotel_telephone':'sum','Hotel_pricerange':'sum','Hotel_starrating':'sum'})
hotel_grouped

,Hotel_address,Hotel_aggregaterating,Hotel_email,Hotel_geo,Hotel_name,Hotel_telephone,Hotel_pricerange,Hotel_starrating
Class,,,,,,,,
Book,0,0,0,0,0,0,0,0
CreativeWork,0,0,0,0,0,0,0,0
Event,0,0,0,0,0,0,0,0
Hotel,269,117,39,122,281,219,131,35
LocalBusiness,0,0,0,0,0,0,0,0
MusicAlbum,0,0,0,0,0,0,0,0
MusicRecording,0,0,0,0,0,0,0,0
Person,0,0,0,0,0,0,0,0
Place,0,0,0,0,0,0,0,0


In [17]:
X = small_books
train_size = int(len(X) * 0.7)
train, test = X[0:train_size], X[train_size:len(X)]
#X_train, y_train, X_test, y_test = iterative_train_test_split(train,test, test_size = 0.2)
#test = test.values
print('Total Counts: %d' % (len(X)))
print('Train Counts: %d' % (len(train)))
print('Testing Counts: %d' % (len(test)))

Total Counts: 43
Train Counts: 30
Testing Counts: 13


In [54]:
X = small_books['filename']
y = small_books[['Book_aggregaterating','Book_author','Book_bookedition','Book_bookformat','Book_datepublished','Book_genre','Book_inlanguage','Book_isbn','Book_name','Book_numberofpages','Book_offers','Book_publisher']]
t_train, y_train, t_test, y_test = train_test_split(X, y, test_size = 0.5)
t_train

41               Book_worklizard.com_September2020.json.gz
26            Book_pspmrsmtumpat.com_September2020.json.gz
11               Book_double-eye.com_September2020.json.gz
16            Book_jillmansell.co.uk_September2020.json.gz
19               Book_marasworld.com_September2020.json.gz
7         Book_carlosruizzafon.co.uk_September2020.json.gz
3                Book_bibliobeat.com_September2020.json.gz
20           Book_memopublishers.com_September2020.json.gz
0            Book_aardvark-books.com_September2020.json.gz
27                 Book_publicvm.com_September2020.json.gz
21                   Book_mrface.com_September2020.json.gz
28                     Book_qpoe.com_September2020.json.gz
33         Book_stepheniemeyer.co.uk_September2020.json.gz
14               Book_hologram.me.uk_September2020.json.gz
4                   Book_bookapy.com_September2020.json.gz
17      Book_librairie-anagramme.com_September2020.json.gz
15               Book_jamesjaffe.com_September2020.json.

In [ ]:
#from skmultilearn.model_selection import iterative_train_test_split
#X = small_books['index']
#y = small_books[['Book_aggregaterating','Book_author','Book_bookedition','Book_bookformat','Book_datepublished','Book_genre','Book_inlanguage','Book_isbn','Book_name','Book_numberofpages','Book_offers','Book_publisher']]
#t_train, y_train, t_test, y_test = iterative_train_test_split(X, y, test_size = 0.2)

In [76]:
#stratified split to make sure that all selected columns/labels are equally distributed in tables used for training and testing
#test size = 0.2

from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
import numpy as np

X = np.array(small_books['filename'])
y = np.array(small_books[['Book_aggregaterating','Book_author','Book_bookedition','Book_bookformat','Book_datepublished','Book_genre','Book_inlanguage','Book_isbn','Book_name','Book_numberofpages','Book_offers','Book_publisher']])

msss = MultilabelStratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=0)

for train_index, test_index in msss.split(X, y):
   print("TRAIN:", train_index, "TEST:", test_index)
   X_train, X_test = X[train_index], X[test_index]
   y_train, y_test = y[train_index], y[test_index]



TRAIN: [ 1  2  3  4  5  6  8  9 10 11 12 13 14 15 17 18 21 22 24 25 27 28 29 30
 31 32 33 35 36 38 39 42] TEST: [ 0  7 16 19 20 23 26 34 37 40 41]
TRAIN: [ 0  1  2  3  6  7  8  9 10 12 13 14 15 16 17 18 19 21 22 24 25 26 28 29
 30 31 32 35 36 38 40 41 42] TEST: [ 4  5 11 20 23 27 33 34 37 39]
TRAIN: [ 1  2  3  4  5  6  7  8  9 10 11 14 16 17 18 19 20 21 22 24 25 26 27 28
 29 31 32 33 34 37 39 40 41 42] TEST: [ 0 12 13 15 23 30 35 36 38]


In [102]:
#test_index
train_df = small_books[small_books['id'].isin(train_index)]
test_df = small_books[small_books['id'].isin(test_index)]
print(test_df['filename'])
print(train_df['filename'])

0        Book_aardvark-books.com_September2020.json.gz
12     Book_george-pelecanos.com_September2020.json.gz
13        Book_gladwellbooks.com_September2020.json.gz
15           Book_jamesjaffe.com_September2020.json.gz
23              Book_onedumb.com_September2020.json.gz
30              Book_rexcity.com_September2020.json.gz
35              Book_suninme.org_September2020.json.gz
36    Book_takethedayoffbook.com_September2020.json.gz
38             Book_toddparr.com_September2020.json.gz
Name: filename, dtype: object
1         Book_americanreadingathome.com_September2020.json.gz
2                 Book_barmatrioshka.com_September2020.json.gz
3                    Book_bibliobeat.com_September2020.json.gz
4                       Book_bookapy.com_September2020.json.gz
5                         Book_books.org_September2020.json.gz
6            Book_capitalsoftworks.co.uk_September2020.json.gz
7             Book_carlosruizzafon.co.uk_September2020.json.gz
8                   Book_clairenor

In [91]:
train_agg = train_df.groupby('Class').agg({'Book_aggregaterating':'sum','Book_author':'sum','Book_bookedition':'sum','Book_bookformat':'sum','Book_datepublished':'sum', 'Book_genre':'sum','Book_inlanguage':'sum','Book_isbn':'sum','Book_name':'sum','Book_numberofpages':'sum','Book_offers':'sum','Book_publisher':'sum'})
train_agg #small_books.groupby('Class').agg({'Book_aggregaterating':'sum','Book_author':'sum','Book_bookedition':'sum'})

,Book_aggregaterating,Book_author,Book_bookedition,Book_bookformat,Book_datepublished,Book_genre,Book_inlanguage,Book_isbn,Book_name,Book_numberofpages,Book_offers,Book_publisher
Class,,,,,,,,,,,,
Book,6,29,3,15,25,13,9,29,34,22,9,19


In [92]:
test_agg = test_df.groupby('Class').agg({'Book_aggregaterating':'sum','Book_author':'sum','Book_bookedition':'sum','Book_bookformat':'sum','Book_datepublished':'sum', 'Book_genre':'sum','Book_inlanguage':'sum','Book_isbn':'sum','Book_name':'sum','Book_numberofpages':'sum','Book_offers':'sum','Book_publisher':'sum'})
test_agg #

,Book_aggregaterating,Book_author,Book_bookedition,Book_bookformat,Book_datepublished,Book_genre,Book_inlanguage,Book_isbn,Book_name,Book_numberofpages,Book_offers,Book_publisher
Class,,,,,,,,,,,,
Book,2,8,1,4,7,4,2,8,9,6,3,7


In [73]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import numpy as np

X = np.array(small_books['filename'])
y = np.array(small_books[['Book_aggregaterating','Book_author','Book_bookedition','Book_bookformat','Book_datepublished','Book_genre','Book_inlanguage','Book_isbn','Book_name','Book_numberofpages','Book_offers','Book_publisher']])

mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=0)

for train_index, test_index in mskf.split(X, y):
   print("TRAIN:", train_index, "TEST:", test_index)
   X_train, X_test = X[train_index], X[test_index]
   y_train, y_test = y[train_index], y[test_index]

TRAIN: [ 0  1  2  3  4  5  7  8  9 10 12 13 15 16 18 19 21 23 24 25 26 27 28 29
 30 31 33 34 35 36 37 38 39 40 42] TEST: [ 6 11 14 17 20 22 32 41]
TRAIN: [ 0  3  4  5  6  7  8  9 11 12 13 14 15 16 17 18 20 21 22 23 24 27 28 30
 31 32 33 34 35 36 37 38 39 41 42] TEST: [ 1  2 10 19 25 26 29 40]
TRAIN: [ 0  1  2  3  4  6  7  8  9 10 11 12 13 14 15 17 18 19 20 21 22 24 25 26
 27 28 29 31 32 33 34 38 40 41] TEST: [ 5 16 23 30 35 36 37 39 42]
TRAIN: [ 0  1  2  4  5  6  7 10 11 12 14 16 17 19 20 22 23 24 25 26 27 29 30 31
 32 33 35 36 37 38 39 40 41 42] TEST: [ 3  8  9 13 15 18 21 28 34]
TRAIN: [ 1  2  3  5  6  8  9 10 11 13 14 15 16 17 18 19 20 21 22 23 25 26 28 29
 30 32 34 35 36 37 39 40 41 42] TEST: [ 0  4  7 12 24 27 31 33 38]


In [22]:
df_grouped = small_books.groupby('Class').agg({'Book_aggregaterating':'sum','Book_author':'sum','Book_bookedition':'sum','Book_bookformat':'sum','Book_datepublished':'sum', 'Book_genre':'sum','Book_inlanguage':'sum','Book_isbn':'sum','Book_name':'sum','Book_numberofpages':'sum','Book_offers':'sum','Book_publisher':'sum'})
df_grouped #small_books.groupby('Class').agg({'Book_aggregaterating':'sum','Book_author':'sum','Book_bookedition':'sum'})

,Book_aggregaterating,Book_author,Book_bookedition,Book_bookformat,Book_datepublished,Book_genre,Book_inlanguage,Book_isbn,Book_name,Book_numberofpages,Book_offers,Book_publisher
Class,,,,,,,,,,,,
Book,8,37,4,19,32,17,11,37,43,28,12,26


In [13]:
small_counts = small.groupby('Class').agg(lambda x : x.sum() if x.dtype=='int64' else x.head(1))
small_counts = small_counts.reset_index()
small_counts.drop(small_counts.iloc[:, 1:9], inplace = True, axis = 1)
#small_counts.to_csv('../../src/data/data/CSV_files/small_coloum_count_overiew_class.csv')
small_counts

,Class,Book_aggregaterating,Book_author,Book_bookedition,Book_bookformat,Book_datepublished,Book_genre,Book_inlanguage,Book_isbn,Book_name,...,Restaurant_menu,Restaurant_name,Restaurant_openinghours,Restaurant_openinghoursspecification,Restaurant_pricerange,Restaurant_servescuisine,Restaurant_telephone,TVEpisode_episodenumber,TVEpisode_name,TVEpisode_partofseries
0,Book,8,37,4,19,32,17,11,37,43,...,0,0,0,0,0,0,0,0,0,0
1,CreativeWork,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Event,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Hotel,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,LocalBusiness,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,MusicAlbum,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,MusicRecording,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Person,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Place,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Product,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#from skmultilearn.model_selection import iterative_train_test_split
#X = small_books['filename']
#y = small_books.columns[small_books.columns.isin(book_col)]
#t_train, y_train, t_test, y_test = iterative_train_test_split(X, y, test_size = 0.2)

In [123]:
#get all columns (relations) which are represented in less than 20 tables
small_table_count = pd.read_csv('../../src/data/data/CSV_files/final_column_table_count.csv')
small_table_count = small_table_count[small_table_count['Count']<20]
small_table_count

,Unnamed: 0,Column,Count
63,63,LocalBusiness_founder,17.0
71,71,LocalBusiness_owns,2.0
106,106,Person_mainentityofpage,16.0
147,147,Product_itemlistelement,18.0
154,154,Product_offerdetails,18.0


In [3]:
#drop all columns (relations) which are represented in less than 20 tables
relevant = pd.read_csv('../../src/data/data/CSV_files/relevant_tables.csv')
small = pd.read_csv('../../src/data/schemafiltereddata/small_tables.csv')
medium = pd.read_csv('../../src/data/schemafiltereddata/medium_tables.csv')
large = pd.read_csv('../../src/data/schemafiltereddata/large_tables.csv')

In [4]:
#drop all columns (relations) which are represented in less than 20 tables
relevant = relevant.drop(['LocalBusiness_founder', 'LocalBusiness_owns','Person_mainentityofpage', 'Product_itemlistelement', 'Product_offerdetails' ], axis=1)
small = small.drop(['LocalBusiness_founder', 'LocalBusiness_owns','Person_mainentityofpage', 'Product_itemlistelement', 'Product_offerdetails' ], axis=1)
medium = medium.drop(['LocalBusiness_founder', 'LocalBusiness_owns','Person_mainentityofpage', 'Product_itemlistelement', 'Product_offerdetails' ], axis=1)
large = large.drop(['LocalBusiness_founder', 'LocalBusiness_owns','Person_mainentityofpage', 'Product_itemlistelement', 'Product_offerdetails' ], axis=1)


In [131]:
relevant.to_csv('../../src/data/data/CSV_files/relevant_tables_more_than_20.csv')
small.to_csv('../../src/data/schemafiltereddata/small_tables_more_than_20.csv')
medium.to_csv('../../src/data/schemafiltereddata/medium_tables_more_than_20.csv')
large.to_csv('../../src/data/schemafiltereddata/large_tables_more_than_20.csv')

In [156]:
#move book train/test tables to folder src/data/schemafiltereddata/TrainTestTables
#create set of tables in test and train
train_set = set(small_books.filename[small_books['id'].isin(train_index)])
test_set = set(small_books.filename[small_books['id'].isin(test_index)])
inputdir = '../../src/data/schemafiltereddata/'+'Book/'
outputdir = '../../src/data/schemafiltereddata/TrainTestTables/Small/'
files = [file for file in os.listdir(inputdir) if file.endswith('.json.gz')]
for file in files:
    # read file
    #df = pd.read_json(os.path.join(inputdir, '{}'.format(file)), compression='gzip', lines=True)
    for file in train_set:
        print(file)
        df = pd.read_json(os.path.join(inputdir, '{}'.format(file)), compression='gzip', lines=True)
        df.to_json(os.path.join(outputdir+'Train/'+file), compression='gzip', orient='records', lines=True)
    for file in test_set:
        print(file)
        df = pd.read_json(os.path.join(inputdir, '{}'.format(file)), compression='gzip', lines=True)
        df.to_json(os.path.join(outputdir+'Test/'+file), compression='gzip', orient='records', lines=True)
    
    
        

Book_qpoe.com_September2020.json.gz
Book_liversite.co.uk_September2020.json.gz
Book_mrface.com_September2020.json.gz
Book_worklizard.com_September2020.json.gz
Book_sheilaoflanagan.com_September2020.json.gz
Book_officecourseware.co.uk_September2020.json.gz
Book_barmatrioshka.com_September2020.json.gz
Book_roadtripusa.com_September2020.json.gz
Book_portablepress.com_September2020.json.gz
Book_whiterabbitbooks.co.uk_September2020.json.gz
Book_studiofun.com_September2020.json.gz
Book_books.org_September2020.json.gz
Book_stepheniemeyer.co.uk_September2020.json.gz
Book_americanreadingathome.com_September2020.json.gz
Book_pdfcloudbook.com_September2020.json.gz
Book_librairie-anagramme.com_September2020.json.gz
Book_capitalsoftworks.co.uk_September2020.json.gz
Book_memopublishers.com_September2020.json.gz
Book_tom-holland.org_September2020.json.gz
Book_davidsedarisbooks.com_September2020.json.gz
Book_jillmansell.co.uk_September2020.json.gz
Book_bibliobeat.com_September2020.json.gz
Book_publicv

In [8]:
small['id']=small.index
small

,filename,Class,num_rows,num_columns,num_columns_selected,num_NAs,percentage_col_NA,percentage_table_NA,sum_NA_sel_col,Book_aggregaterating,Book_author,Book_bookedition,Book_bookformat,Book_datepublished,Book_genre,Book_inlanguage,Book_isbn,Book_name,Book_numberofpages,Book_offers,Book_publisher,CreativeWork_aggregaterating,CreativeWork_alternativeheadline,CreativeWork_articlebody,CreativeWork_author,CreativeWork_commentcount,CreativeWork_copyrightholder,CreativeWork_copyrightyear,CreativeWork_creator,CreativeWork_datecreated,CreativeWork_datemodified,CreativeWork_datepublished,CreativeWork_genre,CreativeWork_headline,CreativeWork_inlanguage,CreativeWork_interactioncount,CreativeWork_interactionstatistic,CreativeWork_interactiontype,CreativeWork_keywords,CreativeWork_mainentityofpage,CreativeWork_name,CreativeWork_offers,CreativeWork_publisher,CreativeWork_version,Event_doortime,Event_duration,Event_enddate,Event_eventattendancemode,Event_eventstatus,Event_location,Event_name,Event_organizer,Event_performer,Event_performers,Event_typicalagerange,Hotel_address,Hotel_aggregaterating,Hotel_email,Hotel_geo,Hotel_name,Hotel_pricerange,Hotel_starrating,Hotel_telephone,LocalBusiness_address,LocalBusiness_addresslocality,LocalBusiness_addressregion,LocalBusiness_aggregaterating,LocalBusiness_areaserved,LocalBusiness_citystatezip,LocalBusiness_contactpoint,LocalBusiness_email,LocalBusiness_faxnumber,LocalBusiness_geo,LocalBusiness_hasmap,LocalBusiness_legalname,LocalBusiness_location,LocalBusiness_name,LocalBusiness_openinghours,LocalBusiness_openinghoursspecification,LocalBusiness_paymentaccepted,LocalBusiness_postalcode,LocalBusiness_pricerange,LocalBusiness_review,LocalBusiness_sameas,LocalBusiness_streetaddress,LocalBusiness_telephone,MusicAlbum_byartist,MusicAlbum_name,MusicAlbum_numtracks,MusicAlbum_track,MusicRecording_byartist,MusicRecording_datecreated,MusicRecording_datepublished,MusicRecording_duration,MusicRecording_genre,MusicRecording_inalbum,MusicRecording_name,Person_additionalname,Person_address,Person_affiliation,Person_birthdate,Person_birthplace,Person_contactpoint,Person_deathdate,Person_email,Person_familyname,Person_faxnumber,Person_gender,Person_givenname,Person_height,Person_homelocation,Person_jobtitle,Person_knowslanguage,Person_makesoffer,Person_memberof,Person_nationality,Person_sameas,Person_telephone,Person_weight,Person_worklocation,Person_worksfor,Place_geo,Place_name,Place_openinghoursspecification,Place_sameas,Place_telephone,Product_additionalproperty,Product_address,Product_aggregaterating,Product_alternatename,Product_audience,Product_author,Product_availability,Product_bestrating,Product_brand,Product_breadcrumb,Product_category,Product_color,Product_condition,Product_datepublished,Product_depth,Product_disambiguatingdescription,Product_geo,Product_gtin,Product_gtin12,Product_gtin13,Product_gtin14,Product_gtin8,Product_height,Product_identifier,Product_isrelatedto,Product_issimilarto,Product_itemcondition,Product_mainentityofpage,Product_manufacturer,Product_material,Product_model,Product_mpn,Product_name,Product_offers,Product_price,Product_pricecurrency,Product_ratingvalue,Product_releasedate,Product_review,Product_reviewcount,Product_reviews,Product_shop-currency,Product_title,Product_weight,Product_width,Product_worstrating,Recipe_aggregaterating,Recipe_author,Recipe_cookingmethod,Recipe_cooktime,Recipe_datemodified,Recipe_datepublished,Recipe_headline,Recipe_ingredients,Recipe_ispartof,Recipe_keywords,Recipe_mainentityofpage,Recipe_name,Recipe_nutrition,Recipe_performtime,Recipe_preptime,Recipe_publisher,Recipe_recipecategory,Recipe_recipecuisine,Recipe_recipeingredient,Recipe_recipeinstructions,Recipe_recipeyield,Recipe_review,Recipe_suitablefordiet,Recipe_totaltime,Restaurant_acceptsreservations,Restaurant_address,Restaurant_aggregaterating,Restaurant_geo,Restaurant_hasmenu,Restaurant_menu,Restaurant_name,Restaurant_openinghours,Restaurant_openinghoursspecification,Restaurant_pricerange,Res

In [3]:
path = '/work-ceph/bizer-tp2021/data_integration_using_deep_learning/src/data/data/CSV_files'
tables = pd.read_csv(path + '/all_files_cleaned.csv')
tables = tables.drop(columns={'Unnamed: 0'})
columns_sorted = tables.groupby('filename')['column_name'].apply(list).reset_index(name='listofcolumns')
columns_sorted.filename = columns_sorted.filename.str.split('_', expand = True)
columns_sorted.index = columns_sorted['filename']
columns_sorted

,filename,listofcolumns
filename,,
Book,Book,"[name, author, isbn, publisher, offers, datepublished, numberofpages, bookformat, inlanguage, ge..."
CreativeWork,CreativeWork,"[headline, author, datepublished, datemodified, name, publisher, mainentityofpage, interactionst..."
Event,Event,"[name, enddate, location, performer, organizer, eventstatus, eventattendancemode, duration, door..."
Hotel,Hotel,"[name, address, telephone, pricerange, geo, aggregaterating, starrating, email]"
LocalBusiness,LocalBusiness,"[name, address, telephone, geo, pricerange, aggregaterating, email, sameas, faxnumber, openingho..."
MusicAlbum,MusicAlbum,"[name, byartist, track, numtracks]"
MusicRecording,MusicRecording,"[name, duration, byartist, inalbum, datepublished, genre, datecreated]"
Person,Person,"[mainentityofpage, jobtitle, birthdate, givenname, familyname, telephone, email, deathdate, addi..."
Place,Place,"[name, geo, telephone, sameas, openinghoursspecification]"


## Split and move small tables

In [5]:
#function to get and move selected train & test tables
#def traintestsplit():
for index in columns_sorted.filename:
    index_df = pd.DataFrame()
    liste = []
    index_df = small[small['Class']==index]
    #display(index_df)
    liste  = list([col for col in small if col.startswith(index+'_')])
    X = np.array(index_df['filename'])
    y = np.array(index_df[liste])
    msss = MultilabelStratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=42)
    for train_index, test_index in msss.split(X, y):
        #print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
	#print(test_index)
    train_set = set(X_train)
    print(train_set)
    test_set = set(X_test)
    print(test_set)
    inputdir = '../../src/data/schemafiltereddata/'+index+'/'
    outputdir = '../../src/data/schemafiltereddata/TrainTestTables/Small/'
    files = [file for file in os.listdir(inputdir) if file.endswith('.json.gz')]  
    for file in files:
        if file in train_set:
        #for file in train_set:
            #print(file)
            df = pd.read_json(os.path.join(inputdir, '{}'.format(file)), compression='gzip', lines=True)
            df.to_json(os.path.join(outputdir+'Train/'+file), compression='gzip', orient='records', lines=True)
        print(file)
        if file in test_set:
        #for file in test_set:
            #print(file)
            df = pd.read_json(os.path.join(inputdir, '{}'.format(file)), compression='gzip', lines=True)
            df.to_json(os.path.join(outputdir+'Test/'+file), compression='gzip', orient='records', lines=True)
        print(file)
print(len([file for file in os.listdir(outputdir+'Test/') if file.endswith('.json.gz')]))


{'Book_americanreadingathome.com_September2020.json.gz', 'Book_george-pelecanos.com_September2020.json.gz', 'Book_carlosruizzafon.co.uk_September2020.json.gz', 'Book_onedumb.com_September2020.json.gz', 'Book_tom-holland.org_September2020.json.gz', 'Book_publicvm.com_September2020.json.gz', 'Book_librairie-anagramme.com_September2020.json.gz', 'Book_writersafterdark.com_September2020.json.gz', 'Book_thegreatbritishbookshop.co.uk_September2020.json.gz', 'Book_worklizard.com_September2020.json.gz', 'Book_liversite.co.uk_September2020.json.gz', 'Book_stepheniemeyer.co.uk_September2020.json.gz', 'Book_toddparr.com_September2020.json.gz', 'Book_gladwellbooks.com_September2020.json.gz', 'Book_cressidacowell.co.uk_September2020.json.gz', 'Book_portablepress.com_September2020.json.gz', 'Book_readomain.com_September2020.json.gz', 'Book_roadtripusa.com_September2020.json.gz', 'Book_capitalsoftworks.co.uk_September2020.json.gz', 'Book_memopublishers.com_September2020.json.gz', 'Book_books.org_Sept

In [10]:
pip install -U ipykernel

     |████████████████████████████████| 125 kB 8.1 MB/s 
  Using cached traitlets-5.1.1-py3-none-any.whl (102 kB)
     |████████████████████████████████| 1.9 MB 39.6 MB/s 
  Using cached ipython-7.29.0-py3-none-any.whl (790 kB)
  Using cached matplotlib_inline-0.1.3-py3-none-any.whl (8.2 kB)
  Attempting uninstall: traitlets
    Found existing installation: traitlets 5.0.5
    Uninstalling traitlets-5.0.5:
      Successfully uninstalled traitlets-5.0.5
  Attempting uninstall: ipython
    Found existing installation: ipython 7.22.0
    Uninstalling ipython-7.22.0:
      Successfully uninstalled ipython-7.22.0
  Attempting uninstall: ipykernel
    Found existing installation: ipykernel 5.3.4
    Uninstalling ipykernel-5.3.4:
      Successfully uninstalled ipykernel-5.3.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 4.2.5 requires pyqt5<5.13, which is no

In [ ]:
if __name__ == '__main__':
    for index in listcolt.filename:
        print(index)
        setcol = set(listcolt.loc[index]['listofcolumns'])
        print(setcol)
        inputdir = '../../src/data/schemafiltereddata/'+index+'/'
        outputdir = '../../src/data/schemafiltereddata/TrainTesttables/Small/'
        train_set = set(small.filename[small['id'].isin(train_index)])
        test_set = set(small.filename[small['id'].isin(test_index)])
        files = [file for file in os.listdir(inputdir) if file.endswith('.json.gz')]
        p = Pool(100)
        #for _ in tqdm(p.imap_unordered(move, [file for file in os.listdir(inputdir) if file.endswith('.json.gz')]), total=len([file for file in os.listdir(inputdir) if file.endswith('.json.gz')])):
        #    pass
        p.map(move, [file for file in os.listdir(inputdir) if file.endswith('.json.gz')])
        p.close()
        #print(len([file for file in os.listdir(outputdir) if file.endswith('.json.gz')]))
        p.join()
        

In [ ]:
index_df = small[small['Class']=='Hotel']
liste  = list([col for col in small if col.startswith('Hotel_')])
X = np.array(index_df['filename'])
y = np.array(small[small.columns.isin(liste)])
print(y)

mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=0)
for train_index, test_index in mskf.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print(test_index)
train_set = set(small.filename[small['id'].isin(train_index)])
test_set = set(small.filename[small['id'].isin(test_index)])

In [23]:
index_df = small[small['Class']=='Hotel']
liste  = list([col for col in small if col.startswith('Hotel_')])
u = np.array(index_df[liste])
u

array([[1, 0, 1, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 1, 1, 0],
       ...,
       [1, 1, 0, ..., 1, 0, 1],
       [1, 0, 0, ..., 0, 0, 1],
       [1, 0, 1, ..., 1, 1, 1]])

In [140]:
train_set = set(small_books.filename[small_books['id'].isin(train_index)])
test_set = set(small_books.filename[small_books['id'].isin(test_index)])
test_set

{'Book_aardvark-books.com_September2020.json.gz',
 'Book_george-pelecanos.com_September2020.json.gz',
 'Book_gladwellbooks.com_September2020.json.gz',
 'Book_jamesjaffe.com_September2020.json.gz',
 'Book_onedumb.com_September2020.json.gz',
 'Book_rexcity.com_September2020.json.gz',
 'Book_suninme.org_September2020.json.gz',
 'Book_takethedayoffbook.com_September2020.json.gz',
 'Book_toddparr.com_September2020.json.gz'}